In [1]:
import pandas as pd
import numpy as np
import entropy
import datetime

In [2]:
trafficMan = pd.read_csv('data/trafficMan.csv', parse_dates=['DATA_AS_OF'])
trafficMan.head()

,Unnamed: 0,ID,SPEED,TRAVEL_TIME,STATUS,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,TRANSCOM_ID,BOROUGH,LINK_NAME
0,0,399,18.01,130,0,2019-04-07 18:59:13,4456452,"40.8011005,-73.92846 40.80151,-73.93066 40.801...",{}_xFzcfbMqAvLTxA`@b@t@Vj@On@u@n@aBp@mBp@{@bAi...,BBBBBBBBBBBBBBBBBBBB,MTA Bridges & Tunnels,4456452,Manhattan,TBB W - FDR S MANHATTAN TRUSS - E116TH STREET
1,6,213,37.28,64,0,2019-04-07 18:59:05,4456450,"40.80069,-73.92878 40.8013005,-73.930181 40.80...",i{_xFzefbMyBvGUlACt@Rj@d@f@z@@`@W\g@bA_DTk@b@i...,BBBBBBBBBBBBBBBBBBBBBBB,MTA Bridges & Tunnels,4456450,Manhattan,FDR N - TBB E 116TH STREET - MANHATTAN TRUSS
2,10,364,22.36,239,0,2019-04-07 18:59:03,4456511,"40.745726,-73.97359 40.745616,-73.97305 40.745...",wcuwF|}nbMTkB?qAwAuRd@qEbBqKnMk]vB{Ir@_EtCwWLaF,BBBBBBBBBBB,NYC_DOT_LIC,4456511,Manhattan,QMT E Manhattan Side - Toll Plaza
3,17,124,37.90,171,0,2019-04-07 18:59:03,4456501,"40.68036,-74.00441001 40.6822,-74.0057201 40.6...",gkhwFp~tbMoJdGkHnCaK`Fat@d`@oe@`NyD|@{L@wHsA{EeB,BBBBBBBBBB,MTA Bridges & Tunnels,4456501,Manhattan,BBT W Toll Plaza - Manhattan Portal
4,19,119,29.20,219,0,2019-04-07 18:59:03,4456502,"40.70631,-74.01501 40.705380,-74.01528 40.7049...",mmmwFx`wbMxDt@rAb@rFv@jCNbJOfEaAre@yMft@y_@`K}...,BBBBBBBBBBBBB,MTA Bridges & Tunnels,4456502,Manhattan,BBT E Manhattan Portal - Toll Plaza


In [3]:
trafficMan['date'] = trafficMan.DATA_AS_OF.dt.date
trafficMan['hour'] = trafficMan.DATA_AS_OF.dt.hour
trafficMan.head()

,Unnamed: 0,ID,SPEED,TRAVEL_TIME,STATUS,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,TRANSCOM_ID,BOROUGH,LINK_NAME,date,hour
0,0,399,18.01,130,0,2019-04-07 18:59:13,4456452,"40.8011005,-73.92846 40.80151,-73.93066 40.801...",{}_xFzcfbMqAvLTxA`@b@t@Vj@On@u@n@aBp@mBp@{@bAi...,BBBBBBBBBBBBBBBBBBBB,MTA Bridges & Tunnels,4456452,Manhattan,TBB W - FDR S MANHATTAN TRUSS - E116TH STREET,2019-04-07,18
1,6,213,37.28,64,0,2019-04-07 18:59:05,4456450,"40.80069,-73.92878 40.8013005,-73.930181 40.80...",i{_xFzefbMyBvGUlACt@Rj@d@f@z@@`@W\g@bA_DTk@b@i...,BBBBBBBBBBBBBBBBBBBBBBB,MTA Bridges & Tunnels,4456450,Manhattan,FDR N - TBB E 116TH STREET - MANHATTAN TRUSS,2019-04-07,18
2,10,364,22.36,239,0,2019-04-07 18:59:03,4456511,"40.745726,-73.97359 40.745616,-73.97305 40.745...",wcuwF|}nbMTkB?qAwAuRd@qEbBqKnMk]vB{Ir@_EtCwWLaF,BBBBBBBBBBB,NYC_DOT_LIC,4456511,Manhattan,QMT E Manhattan Side - Toll Plaza,2019-04-07,18
3,17,124,37.90,171,0,2019-04-07 18:59:03,4456501,"40.68036,-74.00441001 40.6822,-74.0057201 40.6...",gkhwFp~tbMoJdGkHnCaK`Fat@d`@oe@`NyD|@{L@wHsA{EeB,BBBBBBBBBB,MTA Bridges & Tunnels,4456501,Manhattan,BBT W Toll Plaza - Manhattan Portal,2019-04-07,18
4,19,119,29.20,219,0,2019-04-07 18:59:03,4456502,"40.70631,-74.01501 40.705380,-74.01528 40.7049...",mmmwFx`wbMxDt@rAb@rFv@jCNbJOfEaAre@yMft@y_@`K}...,BBBBBBBBBBBBB,MTA Bridges & Tunnels,4456502,Manhattan,BBT E Manhattan Portal - Toll Plaza,2019-04-07,18


In [4]:
frames = []

for col in trafficMan.LINK_NAME.unique():
    temp = trafficMan[trafficMan.LINK_NAME == col].groupby(by='date', as_index=False).count()[['date', 'LINK_NAME']]
    temp['LINK_NAME'] = col
    frames.append(temp)

In [6]:
entrop = []

for col in trafficMan.LINK_NAME.unique():
    ent = []
    print(col)
    temp = trafficMan[trafficMan.LINK_NAME == col]
    
    
    for date in temp.date.unique():
        tempEntr = []
        temp1 = temp[temp.date == date]
#         temp11 = temp1[(temp1.hour < 11)]
#         temp12 = temp1[(temp1.hour > 11) & (temp1.hour < 16)]
#         temp13 = temp1[(temp1.hour > 16) & (temp1.hour < 20)]
#         temp14 = temp1[(temp1.hour > 20)]
        if len(temp1) > 0:
            try:
                tempEntr.append(entropy.sample_entropy(temp1.SPEED.values))
            except IndexError:
                tempEntr.append(np.nan)
        else:
            tempEntr.append(np.nan)
#         if len(temp12) > 0:
#             try:
#                 tempEntr.append(entropy.sample_entropy(temp12.SPEED.values))
#             except IndexError:
#                 tempEntr.append(np.nan)
#         else:
#             tempEntr.append(np.nan)
#         if len(temp13) > 0:
#             try:
#                 tempEntr.append(entropy.sample_entropy(temp13.SPEED.values))
#             except IndexError:
#                 tempEntr.append(np.nan)
#         else:
#             tempEntr.append(np.nan)
#         if len(temp14) > 0:
#             try:
#                 tempEntr.append(entropy.sample_entropy(temp14.SPEED.values))
#             except IndexError:
#                 tempEntr.append(np.nan)
#         else:
#             tempEntr.append(np.nan)
        ent.append(tempEntr)
        
    entrop.append(ent)

TBB W - FDR S MANHATTAN TRUSS - E116TH STREET
FDR N - TBB E 116TH STREET - MANHATTAN TRUSS
QMT E Manhattan Side - Toll Plaza
BBT W Toll Plaza - Manhattan Portal
BBT E Manhattan Portal - Toll Plaza
12th Ave S 57th St - 45th St
11th ave n ganservoort - 12th ave @ 40th st
FDR S 63rd - 25th St
FDR N 25th - 63rd St
FDR S 25th St - Catherine Slip
FDR N Catherine Slip - 25th St
BQE N Atlantic Ave - BKN Bridge Manhattan Side
11th ave s ganservoort - west st @ spring st
12th ave @ 45th - 11 ave ganservoort st
GWB E LOWER LEVEL PLAZA - CBE E LOWER LEVEL AMSTERDAM AVE
BKN Bridge Manhattan Side - FDR N Catherine Slip
FDR S Catherine Slip - BKN Bridge Manhattan Side
West St S Spring St - BBT Manhattan Portal outbound
LINCOLN TUNNEL W CENTER TUBE NY - NJ
LINCOLN TUNNEL W NORTH TUBE NY - NJ
LINCOLN TUNNEL E CENTER TUBE NJ - NY
LINCOLN TUNNEL E SOUTH TUBE - NJ - NY
Westside Hwy S GWB - 57th St
Westside Hwy N 57th St - GWB
BQE N Atlantic Ave - MAN Bridge Manhattan Side
12th Ave N 40th - 57th St
West St

In [7]:
for i in range(len(frames)):
    temp1 = pd.DataFrame(entrop[i])
    temp1['date'] = frames[i].date
    frames[i] = frames[i].merge(temp1, on='date')

In [12]:
LinkEntrop = pd.concat(frames)
LinkEntrop['date'] = pd.to_datetime(LinkEntrop.date)
LinkEntrop = LinkEntrop[LinkEntrop.date > datetime.datetime(2017, 1, 1)]
LinkEntrop.rename(columns={0:'entrop'}, inplace=True)
LinkEntrop.replace([np.inf, -np.inf], np.nan, inplace=True)

In [13]:
LinkEntrop.describe()

,entrop
count,15750.000000
mean,0.688075
std,0.369662
min,-0.000000
25%,0.416497
50%,0.592932
75%,0.877673
max,3.663562


In [14]:
LinkEntrop.to_csv('LinkEntropMan1.csv')

In [24]:
temp5 = pd.read_csv('temp5.csv', parse_dates=['DATA_AS_OF'])
temp5.head()

,DATA_AS_OF,TBB W - FDR S MANHATTAN TRUSS - E116TH STREET_SPEED,FDR N - TBB E 116TH STREET - MANHATTAN TRUSS_SPEED,QMT E Manhattan Side - Toll Plaza_SPEED,BBT W Toll Plaza - Manhattan Portal_SPEED,BBT E Manhattan Portal - Toll Plaza_SPEED,12th Ave S 57th St - 45th St_SPEED,11th ave n ganservoort - 12th ave @ 40th st_SPEED,FDR S 63rd - 25th St_SPEED,FDR N 25th - 63rd St_SPEED,...,FDR S Catherine Slip - BKN Bridge Manhattan Side_SPEED,West St S Spring St - BBT Manhattan Portal outbound_SPEED,LINCOLN TUNNEL W CENTER TUBE NY - NJ_SPEED,LINCOLN TUNNEL W NORTH TUBE NY - NJ_SPEED,LINCOLN TUNNEL E CENTER TUBE NJ - NY_SPEED,LINCOLN TUNNEL E SOUTH TUBE - NJ - NY_SPEED,Westside Hwy S GWB - 57th St_SPEED,Westside Hwy N 57th St - GWB_SPEED,BQE N Atlantic Ave - MAN Bridge Manhattan Side_SPEED,12th Ave N 40th - 57th St_SPEED
0,2017-05-05 10:00:00,20.090833,31.741667,4.193333,26.288094,21.487914,15.379167,8.390000,32.622500,26.668333,...,26.200000,13.800446,24.176046,25.668462,13.050769,7.313846,39.765000,40.596667,14.239167,10.873333
1,2017-05-08 10:00:00,23.950909,30.560000,6.610909,26.288094,1.860000,14.810000,13.256667,43.030000,33.916667,...,30.808333,1.860000,24.176046,30.656667,10.423333,5.934444,38.005833,47.563636,15.015833,12.272500
2,2017-05-12 10:00:00,31.430833,8.233333,4.246667,26.288094,21.487914,35.109167,7.147500,43.391667,16.569167,...,42.407500,13.800446,24.176046,30.675455,11.692727,4.075556,35.109167,28.739167,10.457500,21.540833
3,2017-05-15 10:00:00,35.418333,14.185833,6.991667,26.288094,1.240000,40.854167,12.115000,39.715000,39.870833,...,20.661667,1.240000,24.176046,29.410833,23.665000,9.063333,34.435833,30.504545,14.187500,24.495000
4,2017-05-17 10:00:00,25.425833,8.077500,3.625000,26.288094,1.860000,18.691667,8.028333,43.340833,34.279167,...,5.591667,1.860000,24.176046,29.272222,7.527778,6.700000,18.691667,28.735833,13.515000,10.874167


In [25]:
temp5['date'] = temp5['DATA_AS_OF'].dt.date

In [29]:
temp5.sort_values(by='DATA_AS_OF', inplace=True)

In [55]:
entrop = []

for date in temp5.date.unique():
    
    tempEntrop = []
    for col in temp5.columns[1:-1]:
        temp1 = temp5[temp5.date == date]
        temp = temp1[col].values
        #print(len(temp))
        try:
            tempEntrop.append(entropy.sample_entropy(temp))
        except IndexError:
            tempEntrop.append(np.nan)
        
    entrop.append(tempEntrop)
    
    print(date)

2017-05-04
2017-05-05
2017-05-08
2017-05-09
2017-05-11
2017-05-12
2017-05-15
2017-05-17
2017-05-18
2017-05-19
2017-05-22
2017-05-23
2017-05-24
2017-05-25
2017-05-26
2017-05-29
2017-05-31
2017-06-01
2017-06-02
2017-06-03
2017-06-06
2017-06-07
2017-06-08
2017-06-09
2017-06-13
2017-06-14
2017-06-15
2017-06-16
2017-06-19
2017-06-20
2017-06-21
2017-06-22
2017-06-23
2017-06-27
2017-06-28
2017-06-29
2017-06-30
2017-07-01
2017-07-07
2017-07-12
2017-07-13
2017-07-14
2017-07-15
2017-07-17
2017-07-18
2017-07-19
2017-07-20
2017-07-25
2017-07-26
2017-07-27
2017-08-03
2017-08-09
2017-08-10
2017-08-11
2017-08-12
2017-08-13
2017-08-14
2017-08-15
2017-08-16
2017-08-17
2017-08-18
2017-08-19
2017-08-20
2017-08-21
2017-08-22
2017-08-23
2017-08-24
2017-08-25
2017-08-26
2017-08-27
2017-08-28
2017-08-29
2017-08-30
2017-08-31
2017-09-01
2017-09-02
2017-09-03
2017-09-04
2017-09-05
2017-09-06
2017-09-07
2017-09-08
2017-09-09
2017-09-10
2017-09-11
2017-09-12
2017-09-13
2017-09-14
2017-09-15
2017-09-16
2017-09-17

In [59]:
x = pd.DataFrame(entrop)
x.replace([np.inf, -np.inf], np.nan, inplace=True)
x.count()

0     399
1     397
2     398
3     305
4     337
5     433
6     483
7     449
8     414
9     437
10    453
11    513
12    461
13    444
14    576
15    369
16    439
17    473
18    604
19    536
20    603
21    590
22    402
23    505
24    421
25    380
dtype: int64

In [6]:
for col in temp5.columns[1:]:
    print(entropy.sample_entropy(temp5[col]))

1.4603823658107158
1.4839115859956802
1.0245142800881006
1.2316218166070936
1.1496854442605096
1.1828832887777037
0.9823412371651878
1.4059151475625749
1.6491311735996845
1.4775752943929852
1.5966521338478383
1.1833837112458656
0.9119664238351487
1.507884641975166
0.9338153898906586
1.644180850200892
1.4300279703356404
0.22901566878607982
1.158395336066995
1.2248350307201676
0.8088827135412797
0.9767296004004226
1.0306532761871172
1.0642929273621946
0.3097423875361391
0.6591180340033818


In [10]:
x = np.random.normal(0, 1, 50)
y = np.random.normal(0, 10, 50)

In [61]:
print(entropy.sample_entropy(np.asarray([50,50,50,50,50,100,100,100,100,100])))
print(entropy.sample_entropy(np.asarray([50,100,50,100,50,100,50,100,50,100])))

0.40546510810816444
-0.0


In [47]:
np.asarray([10, 20, 10, 20, 20], dtype=np.float64)

array([10., 20., 10., 20., 20.])

In [48]:
_numba_sampen(x, mm=order, r=0.2)

NameError: name '_numba_sampen' is not defined